In [ ]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('..')
import seaborn as sns
import os
import pandas as pd
from copy import deepcopy
from matplotlib import pyplot as plt
from os.path import join
import numpy as np
import imodelsx.process_results
import qa_questions
import random
import joblib
from tqdm import tqdm
import feature_spaces
fit_encoding = __import__('01_fit_encoding')
import encoding_utils

### Look at examples

In [ ]:
class A:
    use_test_setup = False
    subject = 'UTS03'


args = A()

In [ ]:
story_names_train, story_names_test = fit_encoding.get_story_names(args)

In [ ]:
wordseqs = feature_spaces.get_story_wordseqs(story_names_train)
# chunks = wordseqs['theclosetthatateeverything'].chunks()
# chunk_lens = [len(chunk) for chunk in chunks]

In [ ]:
seed = 43  # 42
ngrams_examples = []
ngram_size = 10
num_examples_per_story = 1
random.seed(seed)
np.random.seed(seed)
for story_name in story_names_train:
    words_list = wordseqs[story_name].data
    ngrams_list = feature_spaces.get_ngrams_list_from_words_list(
        words_list, ngram_size=ngram_size)[ngram_size + 2:]
    ngrams_examples += np.random.choice(ngrams_list,
                                        num_examples_per_story).tolist()

In [ ]:
print('\n'.join(['- ' + ngram for ngram in ngrams_examples]))

### Look at alignment

In [9]:
wordseqs.keys()

dict_keys(['wheretheressmoke', 'fromboyhoodtofatherhood'])

In [34]:
wordseqs = feature_spaces.get_story_wordseqs(['sloth'])
# chunks = wordseqs['theclosetthatateeverything'].chunks()
# chunk_lens = [len(chunk) for chunk in chunks]

In [35]:
ws = wordseqs['sloth']

In [36]:
len(ws.chunks())

452

In [40]:
len(feature_spaces._get_ngrams_list_from_chunks(ws.chunks()))

452

In [32]:
resp = encoding_utils.get_response(['sloth'], args.subject)

In [33]:
resp.shape

(437, 95556)

In [ ]:
ws = wo

In [17]:
len(wordseqs['wheretheressmoke'].data)

1839

In [20]:
len(wordseqs['wheretheressmoke'].chunks()[10:-5])

291

In [28]:
wordseqs['wheretheressmoke'].chunks()


def get_ngrams_list_from_chunks(chunks, num_trs=2):
    ngrams_list = []
    for i, chunk in enumerate(chunks):
        # print(chunks[i - num_trs:i])
        # sum(chunks[i - num_trs:i], [])
        chunk_block = chunks[i - num_trs:i]
        if len(chunk_block) == 0:
            ngrams_list.append('')
        else:
            chunk_block = np.concatenate(chunk_block)
            ngrams_list.append(' '.join(chunk_block))

    return ngrams_list


ngrams_list = get_ngrams_list_from_chunks(
    wordseqs['wheretheressmoke'].chunks(), num_trs=2)

In [29]:
ngrams_list

['',
 '',
 '',
 '',
 '',
 '',
 'i reached over and secretly',
 'i reached over and secretly undid my seatbelt',
 'undid my seatbelt and when his foot hit the',
 'and when his foot hit the brake at the red light i flung open',
 'brake at the red light i flung open the door and i ran',
 'the door and i ran i had no shoes',
 'i had no shoes on i was crying',
 'on i was crying i had no wallet',
 'i had no wallet but i was ok because i had my cigarettes',
 'but i was ok because i had my cigarettes',
 "and i didn't want any part of freedom if i didn't",
 "and i didn't want any part of freedom if i didn't have my cigarettes",
 'have my cigarettes when you live with someone',
 'when you live with someone who has a temper',
 'who has a temper a very bad temper a very',
 'a very bad temper a very very bad temper you learn',
 'very bad temper you learn to play',
 'to play around that you learn',
 "around that you learn this time i'll play possum",
 "this time i'll play possum and next time i'll j

In [ ]:
args

### Load fitted models

In [ ]:
results_dir = '/home/chansingh/mntv1/deep-fMRI/encoding/results_mar30'
experiment_filename = '../experiments/01_fit_encoding.py'

# load the results in to a pandas dataframe
r = imodelsx.process_results.get_results_df(results_dir)
r = imodelsx.process_results.fill_missing_args_with_default(
    r, experiment_filename)
# imodelsx.process_results.delete_runs_in_dataframe(
# r[r.feature_space == 'qa_embedder-10'], actually_delete=True)
cols_varied = imodelsx.process_results.get_experiment_keys(
    r, experiment_filename)
print('experiment varied these params:', cols_varied)

In [ ]:
args = r[(r.feature_space == 'qa_embedder-5') * (r.pc_components == 100) * (r.ndelays == 4)
         ].sort_values(by='corrs_tune_mean', ascending=False).iloc[0]

In [ ]:
args['corrs_test_pc']

In [ ]:
story_names_train, story_names_test = fit_encoding.get_story_names(args)
stim_test_delayed, resp_test = fit_encoding.get_data(args, story_names_test)
stim_train_delayed, resp_train = fit_encoding.get_data(args, story_names_train)